In [15]:
import pandas as pd
import pymupdf
import pdfplumber
import re
import os
import spacy

#focus on results and discussion?

In [4]:
pdf_file_path = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Desktop\GENIEpdfs\Chen-2019.pdf"

everything_file_path = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Desktop\pdfOutputs\pdfextractall.txt"

output_file_path = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Desktop\pdfOutputs\pdfextractdiscussion.txt"

### Extracts everything

In [5]:
doc = pymupdf.open(pdf_file_path)

# Initialize an empty string to store the text
text = ""

# Loop through all pages and extract text
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text += page.get_text("text")  # Extract the text from each page

# Close the PDF
doc.close()

# Write the complete text to a .txt file
with open(everything_file_path, 'w', encoding='utf-8') as file:
    file.write(text)

print(f"Complete text has been saved to {everything_file_path}")

Complete text has been saved to C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Desktop\pdfOutputs\pdfextractall.txt


### Extracts only the Results section

In [6]:
# This is the best result so far

# Function to clean the extracted text, can be greatly improved
def clean_text(text):
    # Remove URLs (http/https/ftp) and email addresses
    text = re.sub(r'http\S+|www\S+|mailto:\S+|\S+@\S+', '', text)
    text = ' '.join(text.split())  # Remove excessive whitespace
    return text

# Function to extract the final section based on headings and stop at any new heading
def extract_final_section(text):
    # Define common section headings to start extraction
    start_headings = [
        "conclusion", "conclusions", "discussion", "results", 
        "summary", "final remarks", "closing thoughts"
    ]
    
    # Regular expression to detect any heading (common format: capitalized words or followed by a colon/newline)
    heading_pattern = re.compile(r'\b[A-Z][A-Za-z\s]+(:|\n)', re.MULTILINE)

    # Case-insensitive search for the start heading
    start_pos = None
    for heading in start_headings:
        match = re.search(rf"\b{heading}\b", text, re.IGNORECASE)
        if match:
            start_pos = match.start()
            break
    
    # If no start heading is found, return a default message or the entire text
    if start_pos is None:
        return "Relevant section not found."

    # Find the next heading after the start position
    end_pos = None
    for match in heading_pattern.finditer(text[start_pos:]):
        heading_text = match.group().strip().lower()
        # Stop at any heading that is **not** a start heading
        if not any(h in heading_text for h in start_headings):
            end_pos = start_pos + match.start()
            break

    # Extract the relevant section, stopping at the first unwanted heading
    if end_pos:
        return text[start_pos:end_pos]
    else:
        return text[start_pos:]  # If no end heading is found, return till the end of the text

# Open the pdf
doc = pymupdf.open(pdf_file_path)

# Initialize an empty string to store the text
text = ""

# Loop through all pages and extract text
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text += page.get_text("text")  # Extract the text from each page

# Clean the extracted text
cleaned_text = clean_text(text)

# Extract only the final section
final_section = extract_final_section(cleaned_text)

# Write the final section to a .txt file
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(final_section)

# Close the pdf
doc.close()

print(f"Final section has been saved to {output_file_path}")

Final section has been saved to C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Desktop\pdfOutputs\pdfextractdiscussion.txt


### Testing Spacy

In [8]:
free_text = r"We also found that the recurrently mutated genes in our study were similar to TCGA and Li et al., and the overlap between these three studies is about 50% (Supplementary Fig. S1B,C). Some cancer-related genes that"

In [11]:
nlp = spacy.load("en")
doc = nlp(free_text)
for token in doc:
    print(token.text, token.pos_, token.dep_)


OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models and if you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [14]:
spacy.info()

{'spacy_version': '3.7.6',
 'location': 'c:\\Users\\maxwell.bicking\\AppData\\Local\\anaconda3\\lib\\site-packages\\spacy',
 'platform': 'Windows-10-10.0.22631-SP0',
 'python_version': '3.10.9',
 'pipelines': {}}

In [ ]:
df["Extract_Date"] = os.root.basename